# Memory

### ConversationBufferMemory 

- 단순히 이전 대화 내용 전체를 저장하는 메모리입니다. 
- 단점은 대화 내용이 길어질수록 메모리도 계속 커지기 때문에 비효율적. 
- 모델 자체에는 메모리가 없기 때문에 우리가 모델에게 요청을 보낼 때 이전 대화 기록 전체를 같이 보내야함. 그래야 모델이 대화 내용을 파악할 수 있음.
- 유저와 AI의 대화가 길어질수록 우리가 모델에게 매번 보내야되는 대화 기록이 길어진다는 것입니다. 상당히 비효율적이고 돈이 많이 든다.

메모리의 종류는 많고 다르지만 API는 같기 때문에 사용이 편함. ( save_context , load_memory_variables)

In [3]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True) 

memory.save_context({"input":"Hi!"}, {"output": "How are you?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi!'), AIMessage(content='How are you?')]}

### ConversationBufferWindowMemory

- 대화의 특정 부분만 저장하자 ! 범위는 우리가 저장할 수 있고 메모리를 특정 사이즈로 저장할 수 있음 

- 챗봇이 전체 대화가 아닌 최근 대화만 기억함

In [4]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4, # 몇개의 메세지만 저장
    )

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

    
add_message(1,1)

In [5]:
add_message(2,2)
add_message(3,3)    
add_message(4,4)

In [6]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='1'),
  AIMessage(content='1'),
  HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4')]}

In [7]:
add_message(5,5)

In [9]:
memory.load_memory_variables({}) # k를 4로 설정해서 제일 오래된 1을 버림

{'history': [HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5')]}

### ConversationSummaryMemory

ConversationSummaryMemory 형태의 메모리는 시간이 지남에 따라 대화의 요약을 만듭니다.

 이것은 시간이 지남에 따라 대화의 정보를 압축하는 데 유용할 수 있습니다.
 
  대화 요약 메모리는 대화가 진행되는 대로 요약하고 현재 요약 내용을 메모리에 저장합니다. 
  
  그러면 이 메모리를 사용하여 지금까지의 대화의 요약 내용을 프롬프트/체인에 삽입할 수 있습니다. 이 메모리는 과거 메시지의 기록을 프롬프트 축어적 보고에 유지하는 것이 토큰을 너무 많이 차지하는 긴 대화에 가장 유용합니다.

In [10]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

def get_history():
    return memory.load_memory_variables({})

add_message("Hi I'm min", "Wow thats good")

In [11]:
add_message("Korea is so pretty", "I wish I could go")

In [12]:
get_history()

{'history': 'The human introduces themselves as Min and the AI responds positively. Min mentions that Korea is pretty and the AI expresses a desire to go there.'}